In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure

In [ ]:
os.listdir
os.mkdir

In [ ]:
def filtro_espacial(imagen, H):
    h = len(H)
    v = len(H[0])
    imagen_ampliada = cv2.copyMakeBorder(imagen, h//2, h//2, v//2, v//2, cv2.BORDER_REPLICATE)
    imagen_ampliada = np.float32(imagen_ampliada)
    imagen_transformada = np.zeros((len(imagen), len(imagen[0]), 3))
    for k in range(3):
        for i in range(h//2, len(imagen) + h//2):
            for j in range(v//2, len(imagen[0]) + v//2):
                nuevo_pixel = np.sum(imagen_ampliada[i - (h//2):i+(h//2) + 1, j - (v//2):j+ (v//2) + 1, k] * H)
                imagen_transformada[i-(h//2), j-(v//2), k] = nuevo_pixel
    
    imagen_transformada = np.uint8(imagen_transformada)
    
    return imagen_transformada

In [ ]:
def filtro_no_lineal(imagen, n, opcion):
    imagen_ampliada = cv2.copyMakeBorder(imagen, int((n-1)/2), int((n-1)/2), int((n-1)/2), int((n-1)/2), cv2.BORDER_REPLICATE)
    imagen_ampliada = np.float32(imagen_ampliada)
    imagen_transformada = np.zeros((len(imagen), len(imagen[0]), 3))
    
    if(opcion == 1):
        for i in range(int((n-1)/2), len(imagen) + int((n-1)/2)):
            for j in range(int((n-1)/2), len(imagen[0]) + int((n-1)/2)):
                for k in range(3):
                    nuevo_pixel = np.min(imagen_ampliada[i - int((n-1)/2):i+int((n-1)/2) + 1, j-int((n-1)/2):j+int((n-1)/2) + 1, k])
                    imagen_transformada[i-int((n-1)/2), j-int((n-1)/2), k] = nuevo_pixel
    
    elif(opcion == 2):
        for i in range(int((n-1)/2), len(imagen) + int((n-1)/2)):
            for j in range(int((n-1)/2), len(imagen[0]) + int((n-1)/2)):
                for k in range(3):
                    nuevo_pixel = np.max(imagen_ampliada[i - int((n-1)/2):i+int((n-1)/2) + 1, j-int((n-1)/2):j+int((n-1)/2) + 1, k])
                    imagen_transformada[i-int((n-1)/2), j-int((n-1)/2), k] = nuevo_pixel
    
    elif(opcion == 3):
        for i in range(int((n-1)/2), len(imagen) + int((n-1)/2)):
            for j in range(int((n-1)/2), len(imagen[0]) + int((n-1)/2)):
                for k in range(3):
                    minimo = np.min(imagen_ampliada[i - int((n-1)/2):i+int((n-1)/2) + 1, j-int((n-1)/2):j+int((n-1)/2) + 1, k])
                    maximo = np.max(imagen_ampliada[i - int((n-1)/2):i+int((n-1)/2) + 1, j-int((n-1)/2):j+int((n-1)/2) + 1, k])
                    nuevo_pixel = (minimo + maximo)/(2)
                    imagen_transformada[i-int((n-1)/2), j-int((n-1)/2), k] = nuevo_pixel
    
    elif(opcion == 4):
        for i in range(int((n-1)/2), len(imagen) + int((n-1)/2)):
            for j in range(int((n-1)/2), len(imagen[0]) + int((n-1)/2)):
                for k in range(3):
                    nuevo_pixel = np.median(imagen_ampliada[i - int((n-1)/2):i+int((n-1)/2) + 1, j-int((n-1)/2):j+int((n-1)/2) + 1, k])
                    imagen_transformada[i-int((n-1)/2), j-int((n-1)/2), k] = nuevo_pixel
   
    imagen_transformada = np.uint8(imagen_transformada)
    
    
    return imagen_transformada

In [ ]:
def crea_datos(carpetasPrueba, tipo, param, pixels_por_celda, num_orientaciones):
    
    carpetasPrueba=os.listdir('Dataset_traffic_sign/')
    H = [[1/9, 1/9, 1/9], [1/9, 1/9, 1/9], [1/9, 1/9, 1/9]]
    X= np.array([])
    y= np.array([])
    i=0

    for clase,subcarpeta in enumerate (carpetasPrueba):

        ini= 'Dataset_traffic_sign/'
        subcarpeta=str(subcarpeta)
        lista_subcarpeta= ini+subcarpeta+'/'


        imagenes_subcarpeta= os.listdir(lista_subcarpeta)

        for imagen in imagenes_subcarpeta:

            titulo= lista_subcarpeta+imagen


            image= cv2.imread(titulo)

            if (tipo == 'no lineal'):
                image_filtrada = filtro_no_lineal(image, 3, param)    
            else:    
                image_filtrada = filtro_espacial(image, param)

            resized = cv2.resize(image_filtrada, (150,150), interpolation = cv2.INTER_AREA) 

            fd, hog_image = hog(resized, orientations=num_orientaciones, pixels_per_cell=(pixels_por_celda,pixels_por_celda),
                        cells_per_block=(1, 1), visualize=True, multichannel=True)

            hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

            fd= np.array(fd)
            if i==0:
                X=np.hstack((X, fd))
                y=np.hstack((y, clase))
                i=i+1
            else: 

                X=np.vstack((X, fd))
                y=np.vstack((y, clase))

    return (X, y)

In [ ]:
def particionar(X, y):
    posiciones= np.arange(X.shape[0])
    posiciones=np.random.permutation(posiciones)
    print(posiciones)
    X=X[posiciones]
    y=y[posiciones]


    numXtrain= round(0.6*X.shape[0])
    numXval = round(0.8*X.shape[0])
    Xtrain= X[:numXtrain,:]
    ytrain= y[:numXtrain,:]
    
    Xval = X[numXtrain:numXval,:]
    yval = y[numXtrain:numXval,:]

    Xtest= X[numXval:,:]
    ytest= y[numXval:,:]

    return(Xtest, Xtrain, Xval, ytest, yval, ytrain)

In [ ]:
from sklearn import naive_bayes
from sklearn import metrics

def predecir_nb(Xtest, Xtrain, Xval, ytest, yval, ytrain):
    #Construir clasificador

    nb = naive_bayes.GaussianNB()

    #entrenar clasificador
    nb.fit(Xtrain, ytrain) 

    #Obtenemos predicciones y accuracies
    pred_train= nb.predict(Xtrain)
    accTrain= metrics.accuracy_score(ytrain,pred_train)

    pred_test= nb.predict(Xtest)
    accTest= metrics.accuracy_score(ytest,pred_test)
    
    pred_val= nb.predict(Xval)
    accVal= metrics.accuracy_score(yval,pred_val)

    return(accTrain*100, accVal*100,accTest*100)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

def predecir_svm(Xtest, Xtrain, Xval, ytest, yval, ytrain):
    
    #construir clasificador
    svm = SVC()
    OVA = OneVsRestClassifier(estimator=svm)
    
    #entrenar clasificador
    OVA.fit(Xtrain, ytrain)
    
    #obtener accuracies
    accTrain= OVA.score(Xtrain, ytrain)

    accTest= OVA.score(Xtest, ytest)
    
    accVal = OVA.score(Xval, yval)
    
    return(accTrain*100, accVal*100, accTest*100)

In [ ]:
from sklearn.linear_model import LogisticRegression

def predecir_log_reg(Xtest, Xtrain, Xval, ytest, yval, ytrain):
    #construir clasificador
    LogReg = LogisticRegression(penalty = 'l2', max_iter = 400, solver = 'newton-cg')
    
    #entrenar clasificador
    LogReg.fit(Xtrain, ytrain)
    
    #obtener accuracies
    accTrain= LogReg.score(Xtrain, ytrain)

    accTest= LogReg.score(Xtest, ytest)
    
    accVal = LogReg.score(Xval, yval)
    
    return(accTrain*100, accVal*100, accTest*100)

In [ ]:
def best_clasificador_nb():
    valores_orientaciones = [3, 5, 8]
    valores_num_pixels = [7, 9, 11]
    bestVal = 0
    bestTrain = 0
    bestTest = 0
    best_params = [-1, -1]
    
    #probamos las diferentes combinaciones de orientaciones y pixeles y nos quedamos con el mejor
    for i in valores_num_pixels:
        for j in valores_orientaciones:
            X, y = crea_datos(carpetasPrueba, 'lineal', H, i, j)
            Xtest, Xtrain, Xval, ytest, yval, ytrain = particionar(X, y)
            accTrain, accVal, accTest = predecir_nb(Xtest, Xtrain, Xval, ytest, yval, ytrain)
            print("Filtro media, numero de pixeles {}, numero de orientaciones {}".format(i, j))
            print("Precision train: {}  Precision validacion: {}  Precision test: {}".format(accTrain, accVal, accTest))
            if (accVal > bestVal):
                bestVal = accVal
                bestTrain = accTrain
                bestTest = accTest
                best_params = [i, j]
    
    return (bestVal, bestTrain, bestTest, best_params)

In [ ]:
def best_clasificador_svm():
    valores_orientaciones = [3, 5, 8]
    valores_num_pixels = [7, 9, 11]
    bestVal = 0
    bestTrain = 0
    bestTest = 0
    best_params = [-1, -1]
    
    #probamos las diferentes combinaciones de orientaciones y pixeles y nos quedamos con el mejor
    for i in valores_num_pixels:
        for j in valores_orientaciones:
            X, y = crea_datos(carpetasPrueba, 'lineal', H, i, j)
            Xtest, Xtrain, Xval, ytest, yval, ytrain = particionar(X, y)
            accTrain, accVal, accTest = predecir_svm(Xtest, Xtrain, Xval, ytest, yval, ytrain)
            print("Filtro media, numero de pixeles {}, numero de orientaciones {}".format(i, j))
            print("Precision train: {}  Precision validacion: {}  Precision test: {}".format(accTrain, accVal, accTest))
            if (accVal > bestVal):
                bestVal = accVal
                bestTrain = accTrain
                bestTest = accTest
                best_params = [i, j]
    
    return (bestVal, bestTrain, bestTest, best_params)

In [ ]:
def best_clasificador_log_reg():
    valores_orientaciones = [3, 5, 8]
    valores_num_pixels = [7, 9, 11]
    bestVal = 0
    bestTrain = 0
    bestTest = 0
    best_params = [-1, -1]
    
    #probamos las diferentes combinaciones de orientaciones y pixeles y nos quedamos con el mejor
    for i in valores_num_pixels:
        for j in valores_orientaciones:
            X, y = crea_datos(carpetasPrueba, 'lineal', H, i, j)
            Xtest, Xtrain, Xval, ytest, yval, ytrain = particionar(X, y)
            accTrain, accVal, accTest = predecir_log_reg(Xtest, Xtrain, Xval, ytest, yval, ytrain)
            print("Filtro media, numero de pixeles {}, numero de orientaciones {}".format(i, j))
            print("Precision train: {}  Precision validacion: {}  Precision test: {}".format(accTrain, accVal, accTest))
            if (accVal > bestVal):
                bestVal = accVal
                bestTrain = accTrain
                bestTest = accTest
                best_params = [i, j]
    
    return (bestVal, bestTrain, bestTest, best_params)

In [ ]:
#Probamos los 4 filtros no lineales de la funcion filtro_no_lineal
#En este momento no tienen sentido porque hemos modificado particionar() y predecir() (era predecir_nb)
#Los resultados están en el documento
for i in range(4):
    X, y = crea_datos(carpetasPrueba, 'no lineal', i+1)
    Xtest, Xtrain, ytest, ytrain = particionar(X, y)
    accTrain, accTest = predecir(Xtest, Xtrain, ytest, ytrain)
    print("filtro no lineal, parametro ", i)
    print(accTrain, accTest)
    
#Probamos el filtro de la media aritmética
#En este momento no tiene sentido porque hemos modificado particionar() y predecir() (era predecir_nb)
#Los resultados están en el documento
H = [[1/9, 1/9, 1/9], [1/9, 1/9, 1/9], [1/9, 1/9, 1/9]]
X, y = crea_datos(carpetasPrueba, 'lineal', H)
Xtest, Xtrain, ytest, ytrain = particionar(X, y)
accTrain, accTest = predecir(Xtest, Xtrain, ytest, ytrain)
print("Filtro lineal")
print(accTrain, accTest)

In [ ]:
bestValNB, bestTrainNB, bestTestNB, best_paramsNB = best_clasificador_nb()

bestValSVM, bestTrainSVM, bestTestSVM, best_paramsSVM = best_clasificador_svm()

bestValRL, bestTrainRL, bestTestRL, best_paramsRL = best_clasificador_log_reg()

In [ ]:
print("El mejor porcentaje de acierto de test con Naive Bayes es {} con parametros numPixeles: {} y numOrientaciones: {}".format(bestTestNB, best_paramsNB[0], best_paramsNB[1]))

print("El mejor porcentaje de acierto de test con SVM es {} con parametros numPixeles: {} y numOrientaciones: {}".format(bestValSVM, best_paramsSVM[0], best_paramsSVM[1]))

print("El mejor porcentaje de acierto de test con Regresion logistica es {} con parametros numPixeles: {} y numOrientaciones: {}".format(bestValRL, best_paramsRL[0], best_paramsRL[1]))

